In [1]:
import json                 #import all tools we'll need
import requests as r
import pandas as pd
import time

In [2]:
url='https://search.ubisoft.com/api/v2/search'             #define all our variables
headers = """Cache-Control: no-cache
Connection: keep-alive
Content-Length: 205
Content-Type: application/json
Host: search.ubisoft.com
Origin: https://www.ubisoft.com
Pragma: no-cache
Referer: https://www.ubisoft.com/en-gb/games
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: same-site
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36"""
headers=dict(i.split(': ') for i in headers.split('\n'))
payload="""{"appId":"f35adcb5-1911-440c-b1c9-48fdc1701c68","fields":["title"],"from":0,"languages":["en-us"],"size":15,"sorts":[{"field":"createdAt","direction":"desc"}],"channels":["ubisoftportal"],"types":["game"]}"""


In [3]:
def get_df0():
    """return 15 lines of data corresponding to 15 first games"""
    
    resp=r.post(url, headers=headers, data=payload)   #collects datas         
    result = resp.json()
    df0 = pd.DataFrame(result)                    #get array with empty boxes except one which contains our dataframe  
    df = pd.DataFrame(df0.iloc[4,3])             #takes the data out of the aforementioned box  
    
    return df

In [4]:
def get_df(df,payload=payload):
    """add 15 more lines of data corresponding to 15 other games"""    
    
    dct = json.loads(payload)        
    while dct['from'] < 200:
        dct['from'] += 15
        payload = str(dct)         #increases the variable used to call the next page   
        
        resp=r.post(url, headers=headers, data=payload)
        result = resp.json()
        df1 = pd.DataFrame(result)                        #get data from next page
        df1 = pd.DataFrame(df1.iloc[4,3])
        
        df = pd.concat([df, df1])         #add the new collected rows to the array
        time.sleep(0.5)                     #wait 1 sec before next request
        
    return df

In [5]:
def clean(df):
    """Data manipulation for a better reading of our datas"""
    
    df= df.reset_index(drop=True)     #Order the index

    flattened_source = pd.DataFrame(dict(df['_source'])).T  #get data stored in dictionary form in a single column
    df = pd.concat([df, flattened_source], axis=1)
    df.drop('_source', axis=1, inplace=True)                #drop the aforementioned column
    
    df.drop(['platformInfoList','gameInfo'], axis=1, inplace=True)        #drop columns containing soup of info which doesn't interest us
    df.drop('_id', axis=1, inplace=True)                    #drop '_id' column, which is the same as 'id' column

    #df = df.rename(columns={'mdmBrand':'brandTeam','boxshotLink':'link_picture1', 'mDMRating':'minumumAge', 'metaKeyword':'keywords','thumbnail':'link_picture2','link':'link-website', 'createdAt':'releaseDate', 'metaDescription':'description'})    
    #df = df[['title','id','releaseDate','sort','minumumAge','genre','link_picture1','link_picture2', 'link-website','developers','brandTeam', 'keywords','description']]
    
    return df

In [6]:
df = get_df0().pipe(get_df).pipe(clean)
df

,_index,_type,_score,sort,id,mdmBrand,mdmInstallment,genre,developers,boxshotLink,...,name,metaDescription,thumbnail,channel,language,link,createdAt,title,metaKeyword,gameSiteLink
0,ubisoftportal_en-us_v2,game,None,[24776928000000],350318,GODS & MONSTERS,Gods & Monsters - Installments,"[Action, Adventure]",[],https://ubistatic19-a.akamaihd.net/ubicomstati...,...,Immortals Fenyx Rising,"Play as Fenyx, a new winged demigod, on a ques...",https://ubistatic19-a.akamaihd.net/ubicomstati...,4,en-us,https://www.ubisoft.com/en-us/game/gods-and-mo...,2755-02-25T00:00:00,Immortals: Fenyx Rising,NaN,NaN
1,ubisoftportal_en-us_v2,game,None,[24745449159000],363230,RIDERS REPUBLIC,Riders Republic -,"[Action, Sports]",[],https://ubistatic19-a.akamaihd.net/ubicomstati...,...,Riders Republic,NaN,https://ubistatic19-a.akamaihd.net/ubicomstati...,4,en-us,https://www.ubisoft.com/en-us/game/riders-repu...,2754-02-25T15:52:39,Riders Republic,NaN,NaN
2,ubisoftportal_en-us_v2,game,None,[24711609600000],363231,Scott Pilgrim - Brand,SCOTT PILGRIM VS THE WORLD - Installment,"[Adventure, Arcade]",[Ubisoft Montreal],https://ubistatic19-a.akamaihd.net/ubicomstati...,...,Scott Pilgrim vs The World: The Game,NaN,https://ubistatic19-a.akamaihd.net/ubicomstati...,4,en-us,https://www.ubisoft.com/en-us/game/scott-pilgr...,2753-01-30T00:00:00,Scott Pilgrim vs The World: The Game,NaN,NaN
3,ubisoftportal_en-us_v2,game,None,[24679123200000],363254,PRINCE OF PERSIA - 205,SANDS OF TIME REMAKE - Installment,"[Action, Adventure, Platformer]",[Ubisoft Pune],https://ubistatic19-a.akamaihd.net/ubicomstati...,...,Prince of Persia Sands of Time Remake,NaN,https://ubistatic19-a.akamaihd.net/ubicomstati...,4,en-us,https://www.ubisoft.com/en-us/game/prince-of-p...,2752-01-20T00:00:00,Prince of Persia: The Sands of Time Remake,NaN,NaN
4,ubisoftportal_en-us_v2,game,None,[24650697600000],363269,AGOS,AGOS: A Game of Space,"[Adventure, Virtual Reality]",[],https://ubistatic19-a.akamaihd.net/ubicomstati...,...,AGOS: A Game of Space,NaN,https://ubistatic19-a.akamaihd.net/ubicomstati...,4,en-us,https://www.ubisoft.com/en-us/game/agos,2751-02-25T00:00:00,AGOS: A Game of Space,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,ubisoftportal_en-us_v2,game,None,[1080000000000],201145,FAR CRY - 108,FAR CRY - 1239,[],[Crytek],https://ubistatic19-a.akamaihd.net/ubicomstati...,...,Far Cry,NaN,https://ubistatic19-a.akamaihd.net/ubicomstati...,4,en-us,https://www.ubisoft.com/en-us/game/,2004-03-23T00:00:00,Far Cry,NaN,NaN
178,ubisoftportal_en-us_v2,game,None,[-2208988800000],168000,TETRIS - 1586,TETRIS NEXT GEN - 4632,"[Arcade, Family, Puzzle, Strategy]",[Ubisoft Montreal],https://ubistatic19-a.akamaihd.net/ubicomstati...,...,Tetris Ultimate,The world’s most-loved puzzle game arrives on ...,https://ubistatic19-a.akamaihd.net/ubicomstati...,4,en-us,https://www.ubisoft.com/en-us/game/tetris-ulti...,1900-01-01T00:00:00,Tetris Ultimate,"tetris, video game, next-gen, xbox1, ps4, tetr...",NaN
179,ubisoftportal_en-us_v2,game,None,[-2208988800000],357908,Film and Television,Rabbids Short Stories,[Movies],[],https://ubistatic19-a.akamaihd.net/ubicomstati...,...,Rabbids Short Stories,Ubisoft Film & Television gave 10 studios and ...,https://ubistatic19-a.akamaihd.net/ubicomstati...,4,en-us,https://www.ubisoft.com/en-us/game/rabbids-sho...,1900-01-01T00:00:00,Rabbids Short Stories,Rabbids Short Stories - Ubisoft Film & Televis...,NaN
180,ubisoftportal_en-us_v2,game,None,[-2208988800000],357482,VR PROJECTS - 2353,VR Rabbids Ride - Installment,"[Action, Family, Arcade]","[LAI Games, Ubisoft]",https://ubistatic19-a.akamaihd.net/ubicomstati...,...,VR Rabbids Ride,Board a rollercoaster and step into the wacky ...,https://ubistatic19-a.akamaihd.net/ubicomstati...,4,en-us,https://www.ubisoft.com/en-us/game/VR-Rabbids-...,1900-01-01T00:00:00,VR Rabbids Ride,"Virtual, Rabbids, Ride, Rollercoaster, VR, pod...",https://laigames.com/games/virtual-rabbids/


In [7]:
df.to_excel('Further_WebScraping_Ubisoft_xx-xx.xlsx', 'xx-xx')